<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/pecan_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np

In [33]:
window_size=60
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

def calc_max_time_diff(df):
    time = pd.to_datetime(df['localminute'],format="%Y-%m-%d %H:%M:%S-%f")
    time_diff = time.diff()
    aa = time_diff.dt.total_seconds()
    #print(aa.max())
    #print(aa.idxmax())
    return aa.max()

In [22]:
WHE_data = pd.read_csv('/content/drive/My Drive/Phd/dataset/1minute_data_newyork.csv')

In [23]:
WHE_data.head()

,dataid,localminute,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,bedroom4,bedroom5,battery1,car1,car2,circpump1,clotheswasher1,clotheswasher_dryg1,diningroom1,diningroom2,dishwasher1,disposal1,drye1,dryg1,freezer1,furnace1,furnace2,garage1,garage2,grid,heater1,heater2,heater3,housefan1,icemaker1,jacuzzi1,kitchen1,kitchen2,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,lights_plugs4,lights_plugs5,lights_plugs6,livingroom1,livingroom2,microwave1,office1,outsidelights_plugs1,outsidelights_plugs2,oven1,oven2,pool1,pool2,poollight1,poolpump1,pump1,range1,refrigerator1,refrigerator2,security1,sewerpump1,shed1,solar,solar2,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,leg1v,leg2v
0,5997,2019-06-24 15:55:00-05,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.003,NaN,-1.036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002,NaN,0.011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.686,NaN,NaN,NaN,0.503,NaN,0.0,NaN,NaN,NaN,122.046,122.846
1,3700,2019-06-24 15:59:00-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.469,0.001,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,0.003,NaN,NaN,NaN,0.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.837,122.618
2,3700,2019-06-24 15:58:00-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513,0.000,NaN,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,0.003,NaN,NaN,NaN,0.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.740,122.503
3,3700,2019-06-24 15:57:00-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.470,0.000,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,0.003,NaN,NaN,NaN,0.006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.682,122.432
4,3700,2019-06-24 15:56:00-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.469,0.001,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012,NaN,0.003,NaN,NaN,NaN,0.005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.754,122.492


In [24]:
fur_data = WHE_data[WHE_data.furnace1.notnull()].loc[:, ['dataid','localminute','solar','furnace1','grid']]
fur_data.solar = fur_data.solar.fillna(0)
fur_data['grid2'] = fur_data['grid']+fur_data['solar']

In [25]:
fur_data.head()

,dataid,localminute,solar,furnace1,grid,grid2
181,558,2019-06-24 16:59:00-05,0.0,0.005,0.611,0.611
182,558,2019-06-24 16:58:00-05,0.0,0.005,0.616,0.616
183,558,2019-06-24 16:57:00-05,0.0,0.005,0.756,0.756
184,558,2019-06-24 16:56:00-05,0.0,0.005,0.862,0.862
185,558,2019-06-24 16:55:00-05,0.0,0.005,0.718,0.718


In [26]:
fur_data = fur_data[fur_data.grid.notnull()]

In [27]:
fur_data.head(20)

,dataid,localminute,solar,furnace1,grid,grid2
181,558,2019-06-24 16:59:00-05,0.0,0.005,0.611,0.611
182,558,2019-06-24 16:58:00-05,0.0,0.005,0.616,0.616
183,558,2019-06-24 16:57:00-05,0.0,0.005,0.756,0.756
184,558,2019-06-24 16:56:00-05,0.0,0.005,0.862,0.862
185,558,2019-06-24 16:55:00-05,0.0,0.005,0.718,0.718
186,558,2019-06-24 16:54:00-05,0.0,0.005,0.720,0.720
187,558,2019-06-24 16:53:00-05,0.0,0.005,0.723,0.723
188,558,2019-06-24 16:52:00-05,0.0,0.005,0.726,0.726
189,558,2019-06-24 16:51:00-05,0.0,0.005,0.736,0.736
190,558,2019-06-24 16:50:00-05,0.0,0.005,0.598,0.598


In [28]:
fur_data.describe()

,dataid,solar,furnace1,grid,grid2
count,3.118500e+06,3.118500e+06,3.118500e+06,3.118500e+06,3.118500e+06
mean,2.896262e+03,6.300138e-01,8.472850e-02,4.611144e-01,1.091128e+00
std,1.675517e+03,1.940275e+00,1.669917e-01,1.908929e+00,1.507821e+00
min,5.580000e+02,-1.270000e-01,-5.000000e-03,-1.961000e+01,0.000000e+00
25%,1.240000e+03,0.000000e+00,5.000000e-03,2.030000e-01,3.040000e-01
50%,2.318000e+03,0.000000e+00,1.000000e-02,4.180000e-01,5.760000e-01
75%,3.996000e+03,4.700000e-02,8.700000e-02,8.300000e-01,1.112000e+00
max,5.679000e+03,2.039600e+01,1.208000e+00,2.037700e+01,2.037700e+01


In [29]:
#Divide the raw data into different groups by house_id
grouped_df = fur_data.groupby(fur_data.columns[0])

In [39]:
whole_dataX = None
whole_dataY = None

In [40]:
for key, item in grouped_df:
  single_house_raw_data = grouped_df.get_group(key)
  single_house_raw_data = single_house_raw_data.sort_values(by='localminute')
  #single_house_raw_data.to_csv('/content/drive/My Drive/Phd/dataset/pecan_newyork/single_house_'+str(key)+'.csv')
  #print(single_house_raw_data.size)
  #aa_max = calc_max_time_diff(single_house_raw_data)
  #if(aa_max>61):
  #  print(str(key))
  #  print(aa_max)
  Whe_data = single_house_raw_data['grid2']
  fre_data = single_house_raw_data['furnace1']
  dataX = create_dataset(Whe_data,window_size)
  app_Y = fre_data[window_size-1:].values.reshape([fre_data.shape[0]-window_size+1,1])
  categorized_Y = np.ones(app_Y.shape)*1
  categorized_Y[[np.where(app_Y<=0.1)[0]],:]=0
  if whole_dataX is None:
    whole_dataX = dataX
    whole_dataY = categorized_Y
  else:
    whole_dataX = np.vstack([whole_dataX,dataX])
    whole_dataY = np.vstack([whole_dataY,categorized_Y])
  print(whole_dataX.shape)

(264901, 60)
(529802, 60)
(794703, 60)
(1059604, 60)
(1324505, 60)
(1589406, 60)
(1854307, 60)
(2119208, 60)
(2384109, 60)
(2649010, 60)
(2913911, 60)
(3117792, 60)


In [41]:
print(whole_dataY.shape)

(3117792, 1)


In [42]:
import pickle
file_name='/content/drive/MyDrive/Phd/dataset/pecan_newyork/pecan_tab_data'
fileObject = open(file_name,'wb')
pickle.dump(whole_dataX,fileObject)
pickle.dump(whole_dataY,fileObject)
fileObject.close()